In [ ]:
import spectral
import matplotlib.pyplot as plt
import numpy as np
import spectral.io.aviris as aviris
import pandas as pd
import pysptools.eea as eea
import pysptools.noise as noise
import pandas as pd
import os

In [ ]:
img = aviris.open("spectal file")

In [ ]:
img

# PURE PIXEL EXTRACTION

In [ ]:
img_data=img.load()

np.set_printoptions(threshold=np.inf, linewidth=140)

img_2d=img_data.reshape((img_data.shape[0]*img_data.shape[1]), img_data.shape[2])

In [ ]:
[purepixels,bands]=eea.eea.PPI(img_2d,10,10)

In [ ]:
bands

# WAVELENGTH EXTRACTION OF PPI

In [ ]:
file_path = 'spectral file'

In [ ]:
column_names =['wavelength_center_position','fwhm_equivalent_gaussian','Uncertainty in wavelength center position',
                           'Uncertainty in FWHM for equivalent gaussian',
                           'channel_number']

In [ ]:
df_wav_ppi = pd.read_csv(file_path, delimiter='\t', header=None, names=column_names)

In [ ]:
wavelength=[]
wavelength = df_wav_ppi.wavelength_center_position

In [ ]:
len(wavelength)

In [ ]:
wavelength[:10]

In [ ]:
max_ppi,min_ppi=max(wavelength),min(wavelength)

In [ ]:
print(max_ppi)
print(min_ppi)

In [ ]:
ppi_wavelen=[]
for wave in wavelength:
    we=float(round(wave))
    ppi_wavelen.append(we)
wave_spec= pd.DataFrame({'wavelength':ppi_wavelen})

# PPI DATAFRAME

In [ ]:
bands

In [ ]:
def read_and_process_file(file_path, wave_spec):
    data = []
    with open(file_path, 'r') as file:
        for line in file:
            if line.strip():  # Check if the line is not empty
                if "Name: " in line:
                    name_value = line.split("Name: ")[1].strip()
                elif "Class: " in line:
                    class_value = line.split("Class: ")[1].strip()
                elif len(line.split()) == 2 and all(value.replace('.', '').isdigit() for value in line.split()):
                    # Check if the line has two elements and both are numeric
                    data.append([float(value) for value in line.split()])

    df = pd.DataFrame(data, columns=['wavelength', f'reflectance#{class_value}#{name_value}'])
    df['wavelength'] = pd.to_numeric(df['wavelength'], errors='coerce') * 1000
    df[f'reflectance#{class_value}#{name_value}'] = pd.to_numeric(df[f'reflectance#{class_value}#{name_value}'], errors='coerce')
    return df[df['wavelength'].isin(wave_spec['wavelength']) == True]


In [ ]:
def merge_files_into_dataframe(directory, wave_spec):
    dfs = pd.DataFrame({'wavelength': wave_spec['wavelength']})

    for filename in os.listdir(directory):
        if filename.endswith('.spectrum.txt'):
            file_path = os.path.join(directory, filename)
            df = read_and_process_file(file_path, wave_spec)
            dfs = pd.merge(dfs, df, on='wavelength', how='left')

    return dfs

In [ ]:
directory = 'spectral library'
result_dataframe = merge_files_into_dataframe(directory, wave_spec)
print(result_dataframe)

In [ ]:
df_transposed = result_dataframe.transpose().reset_index()
df_transposed.set_axis(df_transposed.iloc[0], axis=1, inplace=True)
df_transposed = df_transposed[1:]
df_transposed[['value', 'class', 'sub - class']] = df_transposed['wavelength'].str.split("#",expand=True)

columns_to_drop = ['value', 'wavelength']
df_transposed = df_transposed.drop(columns=columns_to_drop)

filtered_columns = df_transposed.columns[df_transposed.notnull().all()]
filtered_df = df_transposed[filtered_columns]

col=list(filtered_df.columns)[:-2]

filtered_df['class'].value_counts()

In [ ]:
filtered_df.head()

In [ ]:
row_indices_to_print = list(bands) # list of ppi bands

specific_rows = img_2d[row_indices_to_print,:]

df_pppi = pd.DataFrame(specific_rows.T, columns=row_indices_to_print)

df_pppi.insert(0, 'wavelength', ppi_wavelen) #inserting the purepixel wavelengths

selected_df2 = df_pppi[df_pppi['wavelength'].isin(col)]

selecteddf_t=selected_df2.T
selecteddf_t.set_axis(selecteddf_t.iloc[0], axis=1, inplace=True)
selecteddf_t= selecteddf_t[1:]

selecteddf_t

# SPECTRAL LIBRARY DATAFRAME

In [ ]:
def spectral_angle_mapper(spectrum1,spectrum2):

    num = np.dot(spectrum1, spectrum2)

        # Calculate the sum of squares separately for each array
    sum_of_squares_array1 = np.sum(spectrum1 ** 2)
    sum_of_squares_array2 = np.sum(spectrum2 ** 2)

    # Take the product of the sums of squares
    product_of_sums_of_squares = sum_of_squares_array1 * sum_of_squares_array2

    # Calculate the square root of the product
    denom = np.sqrt(product_of_sums_of_squares)
    arccosine_value = np.arccos(num/denom)

     # Assign the result to the variable 'sam'
    sam = arccosine_value

    return sam

In [ ]:
sam = spectral_angle_mapper(filtered_df.iloc[1, :85].values,selecteddf_t.iloc[1,:].values )

In [ ]:
sam

In [ ]:
import pandas as pd

result_df = pd.DataFrame()

for index1, row1 in selecteddf_t.iterrows():

    for index2, row2 in filtered_df.iterrows():

print(result_df)


In [ ]:

num_rows_df1 = len(selecteddf_t)
num_rows_df2 = len(filtered_df)

new_dataframe = pd.DataFrame(index=range(num_rows_df2))



In [ ]:
new_dataframe.head()

""
0
1
2
3
4


In [ ]:
for index1, row1 in selecteddf_t.iterrows():

    for index2, row2 in filtered_df.iterrows():

            sam = spectral_angle_mapper(row2[:85].values, row1.values)

            new_dataframe.at[index2, index1] = sam

In [ ]:
new_dataframe=new_dataframe[1:]

In [ ]:
new_dataframe.shape

In [ ]:
new_dataframe['mineral'] = filtered_df['class'].astype(str) + " _ " + filtered_df['sub - class'].astype(str)

In [ ]:
result= new_dataframe.T
result.columns = result.loc['mineral']
result = result.drop('mineral')


In [ ]:
result.head()

In [ ]:
result_numeric = result.apply(pd.to_numeric, errors='coerce')

result['classification'] = result_numeric.idxmin(axis=1)

result['min_value'] = result_numeric.min(axis=1)

In [ ]:
result